# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np
df_raw = pd.read_csv("admissions.csv")

FileNotFoundError: File b'admissions.csv' does not exist

In [ ]:
df_raw = pd.read_csv("admissions.csv")
df = df_raw.dropna() 
df.head()

## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [ ]:
# frequency table for prestige and whether or not someone was admitted
df_raw.admit.value_counts()

## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [ ]:
dummy_ranks =  pd.get_dummies(df['prestige'], prefix='prestige')
dummy_ranks.columns=['prestige_1','prestige_2','prestige_3','prestige_4']

In [ ]:
dummy_ranks.head()

#### 2.2 When modeling our class variables, how many do we need? 



Answer: We only need three because when they are all 0 it implies the fourth variable

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [ ]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
print (handCalc.head())

In [ ]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
prestige_df=pd.crosstab(index=df_raw['admit'],columns=df_raw['prestige'])
prestige_df.columns=['prestige_1','prestige_2','prestige_3','prestige_4']
prestige_df.index= ["not admitted","admitted"]
prestige_df

#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [ ]:
one_yes=(33/(28+33))/(1-(33/(28+33)))
one_yes

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [ ]:
one_no=(93/(245+93))/(1-(93/(245+93)))
one_no

#### 3.3 Calculate the odds ratio

In [ ]:
one_no/one_yes

#### 3.4 Write this finding in a sentenance: 

Answer: Odds of getting admitted if you aren't in the most prestigous category decrease by 32%

#### 3.5 Print the cross tab for prestige_4

#### 3.6 Calculate the OR 

#### 3.7 Write this finding in a sentence

Answer:

## Part 4. Analysis

In [ ]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
data.head()

We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [ ]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [ ]:
train_cols=data.columns[1:]

#### 4.2 Fit the model

In [ ]:
logit = sm.Logit(data['admit'], data[train_cols])
result = logit.fit()

#### 4.3 Print the summary results

result.summary()

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [ ]:
np.exp(result.params)

In [ ]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
np.exp(conf)

#### 4.5 Interpret the OR of Prestige_2

Answer: we can expect the odds of being admitted to decrease by about 50% if the prestige of a school is 2

#### 4.6 Interpret the OR of GPA

Answer: we can expect the odds of being admitted to increase by about 118% if there is a one unit increase in GPA

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [ ]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """
    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in range(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [ ]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print (gres)
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print (gpas)
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

#### 5.1 Recreate the dummy variables

In [ ]:
combos.head()

In [ ]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
dummy_ranks = pd.get_dummies(combos['prestige'], prefix='prestige')
dummy_ranks.columns = ['prestige_1', 'prestige_2', 'prestige_3', 'prestige_4']
# keep only what we need for making predictions
cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos = combos[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])

#### 5.2 Make predictions on the enumerated dataset

In [ ]:
combos.head()

In [ ]:
combos['admit_pred'] = result.predict(combos[train_cols])
combos.tail(4)

#### 5.3 Interpret findings for the last 4 observations

Answer: 
<br>Observation 396: with a perfect GRE and GPA and prestige of school equal to 1, you have a 73.4% chance of being admitted
<br>Observation 397: with a perfect GRE and GPA and prestige of school equal to 2, you have a 58.2% chance of being admitted
<br>Observation 398: with a perfect GRE and GPA and prestige of school equal to 3, you have a 41.9% chance of being admitted
<br>Observation 400: with a perfect GRE and GPA and prestige of school equal to 4, you have a 36.8% chance of being admitted

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.